In [81]:
import numpy as np
import pandas as pd
import psycopg2
import recordlinkage

import settings

In [82]:
conn = psycopg2.connect(database=settings.DB, user=settings.UN, password=settings.PW,
        host=settings.H, port=settings.PORT)

In [83]:
tables = pd.read_sql_query("""
SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='public'
   AND table_type='BASE TABLE'
   AND table_name not like '%_old'""", conn)

In [84]:
table_names = tables['table_name'].tolist()

In [85]:
table_names

['goe',
 'pcs',
 'scraped_totals',
 'total_scores',
 'judges_with_countries',
 'calls',
 'elt_scores',
 'deductions',
 'judges',
 'competitors']

In [86]:
df_clean = pd.read_sql_query("Select * FROM competitors", conn)

In [87]:
df_clean.sort_values('name').head()

,line_id,season,disc,category,name,country
713,713,SB2009,Ladies,Sr,Abigail PIETERSEN,RSA
907,907,SB2017,Men,Sr,Abzal RAKIMGALIEV,KAZ
933,933,SB2011,Men,Sr,Adam RIPPON,USA
938,938,SB2012,Ladies,Sr,Adelina SOTNIKOVA,RUS
309,309,SB2010,Ladies,Jr,Adelina SOTNIKOVA,RUS


In [88]:
df = df_clean.copy()

In [89]:
df.set_index('line_id', inplace=True)

In [90]:
df['name'] = df['name'].apply(lambda x: x.lower())

In [91]:
comparison_methods = ['jaro','jarowinkler', 'levenshtein','damerau_levenshtein', 'qgram', 'cosine']

In [92]:
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(df)

In [24]:
for method in comparison_methods:
    compare_cl = recordlinkage.Compare()
    compare_cl.string('name', 'name', method=method, label='name_match')

    features = compare_cl.compute(pairs, df)
    
    matches = features[features.name_match >= .75]
    matches.reset_index(inplace=True)
    matches = matches[matches['line_id_1'] != matches['line_id_2']]
    
    matched = matches.merge(df, left_on='line_id_1', right_on='line_id')
    matched = matched.merge(df, left_on='line_id_2', right_on='line_id', suffixes=['_First', '_Second'])
    matched = matched[matched['disc_First'] == matched['disc_Second']]
    
    print(f"Similarity method {method}: {len(matched[matched['name_match'] != 1])} potential matches")
    print(matched[matched['name_match'] != 1].sort_values('name_match', ascending=True).head(20))
    

Similarity method jaro: 555 potential matches
     line_id_1  line_id_2  name_match season_First disc_First category_First  \
326        572        261    0.750000       SB2004        Men             Sr   
125        425         27    0.750000       SB2006     Ladies             Sr   
493        778        440    0.750000       SB2012     Ladies             Sr   
407        453        113    0.750000       SB2014     Ladies             Sr   
515        542        377    0.750000       SB2009        Men             Sr   
561        596        190    0.750000       SB2005     Ladies             Sr   
20         423         18    0.750000       SB2015        Men             Sr   
18         342         18    0.750000       SB2014        Men             Jr   
35         131        129    0.750318       SB2015     Ladies             Sr   
207        853         90    0.750318       SB2017     Ladies             Sr   
204        355         90    0.750318       SB2016     Ladies             

Similarity method levenshtein: 45 potential matches
     line_id_1  line_id_2  name_match season_First disc_First category_First  \
69         662        276    0.750000       SB2008     Ladies             Sr   
95         797        765    0.750000       SB2013        Men             Sr   
51         962        193    0.750000       SB2014        Men             Sr   
50         523        193    0.750000       SB2008        Men             Jr   
71         672        635    0.750000       SB2008     Ladies             Sr   
96         908        765    0.750000       SB2017        Men             Sr   
118        888        423    0.764706       SB2017        Men             Sr   
9          360         18    0.764706       SB2016        Men             Jr   
10         424         18    0.764706       SB2017        Men             Sr   
22         888        342    0.764706       SB2017        Men             Sr   
67         655        238    0.769231       SB2006        Men       

Similarity method qgram: 24 potential matches
     line_id_1  line_id_2  name_match season_First disc_First category_First  \
53         672        635    0.760000       SB2008     Ladies             Sr   
73         787        546    0.777778       SB2013     Ladies             Sr   
45         546        317    0.777778       SB2013     Ladies             Sr   
1          223        219    0.785714       SB2011     Ladies             Sr   
108        940        550    0.800000       SB2012     Ladies             Sr   
66         751        695    0.800000       SB2010     Ladies             Sr   
68         777        692    0.800000       SB2012     Ladies             Sr   
3          276        246    0.833333       SB2016     Ladies             Sr   
102        908        797    0.833333       SB2017        Men             Sr   
79         809        543    0.833333       SB2014     Ladies             Sr   
91         867        824    0.833333       SB2017     Ladies             

In [93]:
compare_cl = recordlinkage.Compare()
compare_cl.string('name', 'name', method='cosine', label='name_match')

features = compare_cl.compute(pairs, df)

In [94]:
matches = features[features.name_match >= .75]
matches.reset_index(inplace=True)
matches = matches[matches['line_id_1'] != matches['line_id_2']]

In [95]:
matched = matches.merge(df, left_on='line_id_1', right_on='line_id')
matched = matched.merge(df, left_on='line_id_2', right_on='line_id', suffixes=['_First', '_Second'])
matched = matched[(matched['disc_First'] == matched['disc_Second'])]

In [96]:
matched[matched['name_match'] != 1].sort_values('name_match', ascending=True).head(30)

,line_id_1,line_id_2,name_match,season_First,disc_First,category_First,name_First,country_First,season_Second,disc_Second,category_Second,name_Second,country_Second
45,508,330,0.750939,SB2008,Ladies,Jr,alexandra kunova,SVK,SB2013,Ladies,Jr,alexandra proklova,RUS
9,330,88,0.750939,SB2013,Ladies,Jr,alexandra proklova,RUS,SB2010,Ladies,Sr,alexandra kunova,SVK
63,681,619,0.759072,SB2008,Men,Sr,igor macypura,SVK,SB2005,Men,Sr,igor matsipura,SVK
76,756,26,0.768706,SB2010,Ladies,Sr,melinda wang,TPE,SB2007,Ladies,Sr,melinda sherilyn wang,TPE
46,508,364,0.769484,SB2008,Ladies,Jr,alexandra kunova,SVK,SB2017,Ladies,Jr,alexandra trusova,RUS
10,364,88,0.769484,SB2017,Ladies,Jr,alexandra trusova,RUS,SB2010,Ladies,Sr,alexandra kunova,SVK
4,777,246,0.769800,SB2012,Ladies,Sr,yuna kim,KOR,SB2015,Ladies,Sr,na hyun kim,KOR
55,578,547,0.782586,SB2004,Men,Sr,evgeni plushenko,RUS,SB2013,Men,Sr,evgeny plyushchenko,RUS
23,425,16,0.783349,SB2006,Ladies,Sr,dan fang,CHN,SB2004,Ladies,Sr,fan zhang,CHN
1,223,219,0.785714,SB2011,Ladies,Sr,sandra kohpon,THA,SB2011,Ladies,Sr,sandra khopon,THA


In [97]:
matched[matched['name_match'] != 1].describe()

,line_id_1,line_id_2,name_match
count,91.000000,91.000000,91.000000
mean,690.571429,355.076923,0.943330
std,219.870626,185.784656,0.083332
min,223.000000,16.000000,0.750939
25%,506.500000,279.000000,0.885634
50%,756.000000,336.000000,1.000000
75%,891.500000,498.500000,1.000000
max,985.000000,849.000000,1.000000


In [117]:
d = {}
check = 0

for index, row in matched.iterrows():
    if not d:
        d[row['line_id_1']] = []
        d[row['line_id_1']].append(row['line_id_1'])
        d[row['line_id_1']].append(row['line_id_2'])
    else:
        for key in d.keys():
            if row['line_id_1'] in d[key]:
                if row['line_id_2'] in d[key]:
                    check = 1
                else:
                    d[key].append(row['line_id_2'])
                    check = 1
            elif row['line_id_2'] in d[key]:
                if row['line_id_1'] in d[key]:
                    check = 1
                else:
                    d[key].append(row['line_id_1'])
                    check = 1
        if check != 1:
            d[row['line_id_1']] = []
            d[row['line_id_1']].append(row['line_id_1'])
            d[row['line_id_1']].append(row['line_id_2'])
        check = 0

In [99]:
id_pair = {}
for key in d.keys():
    for line_id in d[key]:
        id_pair[line_id] = key

In [100]:
id_pair_df = pd.DataFrame(pd.Series(id_pair))
id_pair_df.reset_index(inplace=True)
id_pair_df.rename(columns={'index': 'line_id', 0: 'master_competitor_id'}, inplace=True)

In [101]:
df_final = df_clean.merge(id_pair_df, on='line_id', how='outer')

In [102]:
df_final['master_competitor_id'] = df_final.apply(lambda x: int(x['line_id']) if np.isnan(x['master_competitor_id']) 
                                                 else int(x['master_competitor_id']), axis=1)

In [103]:
df_final = df_final.merge(df_final[['line_id', 'name']], left_on='master_competitor_id', right_on='line_id', suffixes=['', 'x'])

In [104]:
df_final.drop('line_idx', axis=1, inplace=True)
df_final.rename(columns={'namex': 'master_competitor_name'}, inplace=True)

In [108]:
df_final[df_final['name'] != df_final['master_competitor_name']]

,line_id,season,disc,category,name,country,master_competitor_id,master_competitor_name
4,4,SB2007,Men,Sr,Robert MCNAMARA,AUS,195,Robert McNAMARA
17,16,SB2004,Ladies,Sr,Fan ZHANG,CHN,425,Dan FANG
26,24,SB2017,Ladies,Sr,Anita OESTLUND,SWE,874,Anita ÖSTLUND
28,25,SB2017,Men,Sr,Junhwan CHA,KOR,278,Jun Hwan CHA
31,26,SB2007,Ladies,Sr,Melinda Sherilyn WANG,TPE,756,Melinda WANG
43,37,SB2004,Ladies,Sr,Jaqueline BELENYESIOVA,SVK,587,Jacqueline BELENYESIOVA
100,88,SB2010,Ladies,Sr,Alexandra KUNOVA,SVK,330,Alexandra PROKLOVA
102,364,SB2017,Ladies,Jr,Alexandra TRUSOVA,RUS,330,Alexandra PROKLOVA
103,508,SB2008,Ladies,Jr,Alexandra KUNOVA,SVK,330,Alexandra PROKLOVA
116,100,SB2011,Ladies,Sr,Romy BÜHLER,SUI,511,Romy BUEHLER


In [116]:
matched[['line_id_1', 'line_id_2']].groupby('line_id_1').count().sort_values('line_id_2', ascending=False)

,line_id_2
line_id_1,
508,3
362,2
787,2
950,2
777,2
901,2
833,1
834,1
836,1
